In [1]:
from tqdm import tqdm
import pandas as pd
import constants as c
from importlib import reload

reload(c)

<module 'constants' from 'g:\\Shared drives\\Data\\Analysis\\Geography Conversion Tables\\src\\constants.py'>

- Cleaning column and row names

In [2]:
# removes the (NSW) from the postcode
def clean_postcode_name(postcode):
    return(postcode.split(', ')[0].split(' ')[0])

def clean_RA_name(RA):
    return(RA.split('(')[0])

def load_weighting_table(geo1, geo2):
    df = pd.read_excel(c.geography_weightings[geo1][geo2], **c.read_options_lprw)
    if geo1 == 'POA':
        df.rename(clean_postcode_name, axis='index', inplace=True)
    if geo2 == 'RA':
        df.rename(clean_RA_name, axis='columns', inplace=True)
    return df[df.columns[1:-1]]

In [3]:
# Loads the geography update, i.e. Postcodes 2016 -> 2021
def load_geo_update_df(geo, Y1, Y2):
    return pd.read_csv(c.geography_updates[geo][Y1 + '-' + Y2])

# With a duplicate, chooses the 2021 Postcode 
def geo_update_sort_drop(df, geo, Y):
    df1 = df[df['RATIO_FROM_TO'] == 1.0]
    df2 = df[df['RATIO_FROM_TO'] != 1.0]
    df2 = df2.sort_values(by='RATIO_FROM_TO', ascending=False)
    df2 = df2.drop_duplicates(subset= geo + '_NAME_'+ Y, keep='first')
    return pd.concat([df1,df2])

In [7]:
def collapse_transform_table(table, geo1, geo2):
    geo1s, geo2s, households = [], [], []
    geo1_name = c.geo_names[geo1]
    geo2_name = c.geo_names[geo2]
    for col in table.columns:
        subset = table[table[col] > 0][col]
        geo1s += list(subset.index)
        households += list(subset)
        geo2s += list([col])*len(subset)

    geo1_totals = table.sum(axis='columns')
    geo1_totals = geo1_totals.reset_index(name=geo1_name + ' Totals').rename({'index': geo1_name}, axis='columns')

    geo2_totals = table.sum(axis='rows')
    geo2_totals = geo2_totals.reset_index(name=geo2_name + ' Totals').rename({'index': geo2_name}, axis='columns')
    df_new = pd.DataFrame({geo1_name: geo1s, geo2_name: geo2s, 'Rented Households': households})

    df_new = df_new.merge(geo1_totals, on=geo1_name, how='left')
    df_new = df_new.merge(geo2_totals, on=geo2_name, how='left')

    df_new = df_new.sort_values(by=[geo1_name])
    df_new[geo1_name + ' Fraction'] = df_new['Rented Households']/df_new[geo1_name + ' Totals']
    df_new[geo2_name + ' Fraction'] = df_new['Rented Households']/df_new[geo2_name + ' Totals']

    for col in ['Rented Households', geo1_name + ' Totals', geo2_name + ' Totals']:
        df_new[col] = df_new[col].astype(int)

    return(df_new)
    

Update the 2016 geographies to 2021

In [5]:
def update_geos(df, df_geo1, df_geo2, geo1, geo2, Y1, Y2):
    geo1_name, geo2_name = c.geo_names[geo1], c.geo_names[geo2]
    df = df.merge(df_geo1, left_on=geo1_name, right_on=geo1 + '_NAME_' + Y1, how='left')
    df.rename({'RATIO_FROM_TO': geo1 + '_' + Y1 + '_' + Y2 + '_RATIO'}, axis='columns', inplace=True)
    df = df.merge(df_geo2, left_on=geo2_name, right_on=geo2 + '_NAME_' + Y1, how='left')
    df.rename({'RATIO_FROM_TO': geo2 + '_' + Y1 + '_' + Y2 + '_RATIO'}, axis='columns', inplace=True)

    cols = [geo1 + '_NAME_2021', geo2 + '_NAME_2021', 'Rented Households', geo1_name + ' Totals', geo2_name + ' Totals',
       geo1_name + ' Fraction', geo2_name + ' Fraction', geo1 + '_NAME_2016', geo2 + '_NAME_2016', 
       geo1 + '_' + Y1 + '_' + Y2 + '_RATIO', geo2 + '_' + Y1 + '_' + Y2 + '_RATIO'] 
    df = df[cols]

    df.rename({geo1 + '_NAME_' + Y2: geo1_name, geo2 + '_NAME_' + Y2: geo2_name}, axis='columns', inplace=True)
    #Drop missing geos
    df = df[(~df[geo1_name].isna()) & (~df[geo2_name].isna())]

    return(df)


In [6]:
table_poa_sa4 = load_weighting_table('POA', 'SA4')
table_poa_lga = load_weighting_table('POA', 'LGA')
table_poa_ced = load_weighting_table('POA', 'CED')
table_poa_sed = load_weighting_table('POA', 'SED')
table_poa_ra = load_weighting_table('POA', 'RA')
table_poa_gccsa = load_weighting_table('POA', 'GCCSA')

In [66]:
df_t = load_geo_update_df('POA', '2016', '2021')
df_poa_2016_to_2021 = geo_update_sort_drop(df_t, 'POA', '2016')

df_t = load_geo_update_df('LGA', '2016', '2021')
df_lga_2016_to_2021 = geo_update_sort_drop(df_t, 'LGA', '2016')

df_t = load_geo_update_df('SA4', '2016', '2021')
df_sa4_2016_to_2021 = geo_update_sort_drop(df_t, 'SA4', '2016')

df_t = load_geo_update_df('SED', '2016', '2021')
df_sed_2016_to_2021 = geo_update_sort_drop(df_t, 'SED', '2016')

df_t = load_geo_update_df('CED', '2016', '2021')
df_ced_2016_to_2021 = geo_update_sort_drop(df_t, 'CED', '2016')

df_t = load_geo_update_df('GCCSA', '2016', '2021')
df_gccsa_2016_to_2021 = geo_update_sort_drop(df_t, 'GCCSA', '2016')

df_t = collapse_transform_table(table_poa_lga, 'POA', 'LGA')
df_poa_lga = update_geos(df_t, df_poa_2016_to_2021, df_lga_2016_to_2021, 'POA', 'LGA', '2016', '2021')

df_t = collapse_transform_table(table_poa_sa4, 'POA', 'SA4')
df_poa_sa4 = update_geos(df_t, df_poa_2016_to_2021, df_sa4_2016_to_2021, 'POA', 'SA4', '2016', '2021')

df_t = collapse_transform_table(table_poa_sed, 'POA', 'SED')
df_poa_sed = update_geos(df_t, df_poa_2016_to_2021, df_sed_2016_to_2021, 'POA', 'SED', '2016', '2021')

df_t = collapse_transform_table(table_poa_ced, 'POA', 'CED')
df_poa_ced = update_geos(df_t, df_poa_2016_to_2021, df_ced_2016_to_2021, 'POA', 'CED', '2016', '2021')

df_t = collapse_transform_table(table_poa_gccsa, 'POA', 'GCCSA')
df_poa_gccsa = update_geos(df_t, df_poa_2016_to_2021, df_gccsa_2016_to_2021, 'POA', 'GCCSA', '2016', '2021')

df_poa_ra = collapse_transform_table(table_poa_ra, 'POA', 'RA')

In [77]:
to_save = {'LGA': df_poa_lga, 'SA4': df_poa_sa4, 'SED': df_poa_sed, 'CED': df_poa_ced, 'GCCSA': df_poa_gccsa, 'RA': df_poa_ra}
for geo, df in to_save.items():
    f_name = c.processed_data_dir + f'POA_{geo}_Rented_Weightings.csv'
    df.sort_values(by='Postcode').to_csv(f_name, index=False)